# Preprocesamiento de datos
En el mundo real, los datos que obtengamos pueden ser erróneos porque estarán incompletos, duplicados o no se podrán usar como tal (machine learning usa sólo **números**)  

Veremos cómo afrontar cada situación con el dataset [Iris Dataset](https://www.kaggle.com/uciml/iris) de kaggle,  está en formato .csv cuyo nombre viene de *Comma Separated Values* o Valores Separados por Comas.

En realidad es sólo texto plano que representa a una tabla. La primera fila tiene los nombres de columnas, separados por comas, y las filas restantes, los valores que corresponden, separados por comas.

Cargaremos el archivo usando el método `read_csv` librería `pandas` 🐼  esto resultará en un objeto DataFrame, cuyo el método head nos permite ver las primeras filas de este.

In [ ]:
import pandas as pd

dataframe = pd.read_csv('./datasets/Iris_mod.csv')
dataframe.head()

Como puedes ver, tenemos un dataset con las medidas de largo y ancho del sépalo y el pétalo de una flor de iris, cuya especie se menciona en la última columna.

También tenemos la columna Id pero no es necesaria, podemos eliminarla con el método `drop` que recibe una lista de las columnas a eliminar.

In [ ]:
dataframe = dataframe.drop(columns=['Id'])

¿Cuántas especies tenemos? usemos la función `nunique` para contar los valores únicos de esa columna, podemos acceder a ella como en un diccionario.

In [ ]:
dataframe['Species'].nunique()

Esto **ignora** los valores **nulos**, para ver cuántos son usamos el método `isnull` que nos devuelve True o False si algún dato es **nulo**, veamos las primeras filas usando `head`

In [ ]:
dataframe.isnull().head()

Pero sería más util tener el total de nulos, para esto sumaremos cada columna pues cada True contará como 1 y cada False como 0

In [ ]:
dataframe.isnull().sum()

## Valores nulos
Tenemos exactamente un valor nulo en cada columna ¿Concidencia?  
No, en realidad modifiqué el dataset con propósitos educativos 😄 añadí una fila de nulos que podemos verla usando `tail`

In [ ]:
dataframe.tail(2)

Estos aparecen como **NaN** siglas de *Not a Number* o **No es un Número**, para tratarlos podemos **eliminar** las **filas** o **columnas** que contengan algún nulo o **reemplazarlos** con la **media** o la **moda** de su columna.

Antes de intentar cualquiera crearemos una **copia** de nuestro dataframe usando `copy` para preservar el original, si no usas copy sólo tendrás una referencia al objeto original.

In [ ]:
df = dataframe.copy()

### Eliminar filas
Es recomendable cuando tenemos muchos datos y pocos nulos, pues no perdemos demasiada información.

Usaremos el método `dropna` que recibe el parámetro `axis` o eje, que corresponde con la dimensión, `0` o `'index'` en el caso de las **filas**.

In [ ]:
df = df.dropna(axis=0)
df.tail()

### Eliminar columnas
La usamos si esta no es importante o tenemos muchos nulos, pues no podíamos recuperar información.

Sólo cambiamos`axis` a `1` o `'columns'` para el caso de las **columnas**.

In [ ]:
df = dataframe.copy()
df = df.dropna('columns')
df.tail()

$\cdots$  
Recuerda siempre que **un sólo nulo basta**  para eliminar **toda** la fila o columna 😉

### Reemplazar con la media
La media es el promedio de la columna, por supuesto sólo funciona con los valores numéricos, por esto no seleccionaremos la columna **Species** usando `iloc` esto nos permite seleccionar por indices al estilo de numpy, y podemos usar slices.

In [ ]:
# todas las filas, de la primera a la penúltima columna
df = dataframe.iloc[:, :-1]
df.tail(3)

sklearn tiene el objeto `SimpleImputer` para reemplazar valores, nulos por defecto, debemos indicarle la `strategy` o estrategia para reemplazar los valores. 

In [ ]:
from sklearn.impute import SimpleImputer

df = dataframe.iloc[:, :-1]
imp = SimpleImputer(strategy='mean')

df_inputed_mean = imp.fit_transform(df)
df_inputed_mean[-1]

Podemos comprobar que sean los valores correctos usando el método `mean`

In [ ]:
df.mean()

### Reemplazar con la moda
La moda es el valor que más se repite, usaremos también `SimpleImputer` cambiando la `strategy`

In [ ]:
df = dataframe.copy()

imp = SimpleImputer(strategy='most_frequent')

df_inputed_mode = imp.fit_transform(df)
df_inputed_mode[-1]

Y podemos comprobarlo con `mode`

In [ ]:
df.mode()

El dataset tiene 50 filas de las 3 especies, `SimpleImputer` sólo tomó la primera que encontró

## Valores no numéricos
Para tratarlos es necesario codificarlos, podríamos asignar 1, 2 o 3 a cada especie, pero esto implica que hay un patrón u orden escondido, el modelo podría entender algo así: 3 > 2 > 1.

En su lugar debemos indicar que son valores de diferente **categoría** con un código que indica algo como
> Iris setosa y no otra cosa

Tendremos **una columna por categoría**, marcando 1 a la que pertenece y 0 a las demás, esto se conoce como **One-hot encoding** y podemos obtenerla con el método `get_dummies` de pandas 🐼

In [ ]:
df_one_hot = pd.get_dummies(df)
df_one_hot.head()

Por defecto actuará con todos los valores con el "dtype" (data type) `object`, pero puedes tener un dataset con categorías en forma de números, asegurate de informarte sobre las features del dataset, podrías encontrar la información en la [fuente](https://www.kaggle.com/uciml/iris) del dataset.

Puedes verificar los `dtypes` de cada columna con ese atributo:

In [ ]:
df.dtypes

También podemos usar `OneHotEncoder` de sklearn, pero sólo enviaremos las columnas que queramos transformar y no admite que los datos tengan valores nulos.  
Por defecto retorna una [matriz dispersa](https://es.wikipedia.org/wiki/Matriz_dispersa) pero podemos cambiarlo cambiando el parámetro `sparse` a falso.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# sólo la última columna
df_species = df_inputed_mode[:, [-1]]

one_hot = OneHotEncoder(sparse=False)
df_one_hot = one_hot.fit_transform(df_species)
df_one_hot[:5]

Si tuviéramos sólo 2 categorías no es necesario agregar 2 columnas, 0 y 1 ya indican la presencia o no de la feature.

Con todo lo aprendido ya vamos dominando las bases del machine learning 😉
Es más, ahora la máquina puede aprender para **distinguir** categorías o **clases**.

Ten las medídas de sépalo, pétalo y dime ¿es setosa u otra cosa?  
$\cdots$

Responderá, cuando aprenda [clasificación](7_clasificacion.ipynb).